import tweepy
import json

class StdOutListener(StreamListener):
    def on_status(self, status):
        print(status.text)
        return True
    def on_error(self, status_code):
        if status_code == 420:
            return False

In [43]:
import pyodbc
conn = pyodbc.connect(r'DSN=starbucks')
curr= conn.cursor()

In [53]:
from __future__ import absolute_import, print_function
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from sqlalchemy.exc import ProgrammingError
import tweepy
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import math, re, string, requests, json

analyzer = SentimentIntensityAnalyzer()
def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    print('{:<40} {}'.format(sentence, str(snt)))

languages = ["English", "French", "German", "Spanish", "Italian", "Russian", "Japanese", "Arabic", "Chinese"]
language_codes = ["en", "fr", "de", "es", "it", "ru", "ja", "ar", "zh"]


apikey = 'lQr03iVJ9Rs3kVBOlZt07ak7q'
apisecret ='mght8IbXaNJ2au57BabPZAqIFh4USJJF4V5GTXGPKdbVCdhd3u'
accesstoken = '887695854076915712-bIE18SRELRjCFp5x0iq469G2vb02HRw'
tokensecret = 'jEJREL1NQhrI0galiaGqDIf9hu6D5mEo5Hjj1suH45YIn'

class StdOutListener(StreamListener):
    def on_status(self, status):
        print(status.text)
        #if status.retweeted:
            #return
        to_lang="en"
        from_lang=language_codes[status.text]
        if (from_lang == "en") or (from_lang == "en-US"):
            translation = sentence
            translator_name = "No translation needed"
        else:
            api_url = "http://mymemory.translated.net/api/get?q={}&langpair={}|{}".format(status.text, from_lang, to_lang)
            hdrs ={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
            'Accept-Encoding': 'none',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive'}
            response = requests.get(api_url, headers=hdrs)
            response_json = json.loads(response.text)
            translation = response_json["responseData"]["translatedText"]
            translator_name = "MemoryNet Translation Service"
            vs = analyzer.polarity_scores(translation)
            sent = str(vs['compound'])
        
        status.text = str.replace(status.text, '\'', '\'\'')
        q= 'insert into starbucks(\
        id_str, created,Uk_time, text, fav,name, followers, sentiment)\
        values (\'%s\',\'%s\', getdate(),\'%s\',\'%s\',\'%s\',\'%s\', \'%s\')'\
        %(status.id_str, status.created_at, status.text, status.favorite_count, status.user.screen_name, status.user.followers_count, sent)
        
        
        
        
        try:
            curr.execute(q)
            conn.commit()
            
        except:
            print(q)
        #('insert into starbucks(\
        #id_str, created,text, fav,name, description, loc, user_created, follower)\
        #values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'\
        #%(status.id_str, status.created_at, status.text, status.favorite_count, status.user.screen_name, status.user.description, status.user.location, status.user.created_at, status.user.followers_count))
        
        
        #print ('hi my name is %s, and my age is %s' % (name,age))
        #curr.execute():
        #id_str = status.id_str
        #created = status.created_at
        #text = status.text
        #fav = status.favorite_count
        #name = status.user.screen_name
        #description = status.user.description
        #loc = status.user.location
        #user_created = status.user.created_at
        #followers = status.user.followers_count

    #conn = pyodbc.connect(r'DSN=starbucks')
    #curr= conn.cursor()

        #table = db['myTable']

               
    def on_error(self, status_code):
        if status_code == 420:
            return False

if __name__ == '__main__':
    l = StdOutListener()
    auth = tweepy.OAuthHandler(apikey, apisecret)
    auth.set_access_token(accesstoken, tokensecret)
    api = tweepy.API(auth)
    stream = Stream(auth, l)
    stream.filter(track=['starbucks',])

The Secret to Starbucks' Hiring Success — Discovery @ Gemnote https://t.co/OWBaohiRNa https://t.co/j5p2MOEzWW


TypeError: list indices must be integers or slices, not str

In [34]:
pwd

'C:\\Users\\Student\\Notebooks'

In [35]:
name = "ali"
age = 12
gender = "male"

In [43]:
print ('hi my name is %s, and my age is %s' % (name,age))

hi my name is ali, and my age is 12


In [44]:
status.id_str

NameError: name 'status' is not defined